In [395]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
%matplotlib inline
sns.set()
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, learning_curve, train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.feature_selection import RFECV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils import shuffle
# loading data
df_train = pd.read_csv("2-1.csv")
df_test = pd.read_csv("2-2.csv")
df_data = df_train.append(df_test)

In [396]:
df_data = shuffle(df_data)
df_data = df_data.reset_index()
del df_data['index']


In [397]:
#cut data
df_train = df_data.loc[:159]
df_test = df_data.loc[160:]

In [398]:
print(df_train.shape)
print(df_test.shape)

(160, 202)
(22, 202)


In [399]:
df_data

,代號,一個月累計漲跌幅,1,2,3,4,5,6,7,8,...,193,194,195,196,197,198,199,200,201,202
0,3231,4.45,284.00,966.00,284.00,19.2,17.0,16.80,20.90,12.30,...,0.98,15040,18841↘,19625↘,27961↘,-0.0188,-0.0145,-0.0114,0.0425,0.1670
1,1795,-7.23,24.50,192.00,24.50,54.1,0.7,117.00,159.00,78.90,...,0.59,2366,934↗,841↗,1373↘,-0.0068,-0.0036,-0.0211,-0.0315,0.0045
2,2496,-5.11,1.75,11.00,1.75,29.2,18.4,21.50,36.60,12.40,...,0.44,56,51.7↗,55.9↘,76.1↘,0.0177,0.0069,-0.0128,-0.0655,-0.0221
3,3481,0.75,971.00,788.00,971.00,17.6,13.8,14.90,18.50,11.60,...,1.22,67742,113706↘,117159↘,115465↘,-0.0165,-0.0223,-0.0254,0.0587,0.1270
4,3189,23.00,45.10,372.00,45.10,19.9,15.8,33.30,41.90,24.50,...,3.26,24180,19070↗,17996↘,15093↗,0.0049,0.0444,0.1240,0.2930,0.4860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,8291,-4.93,7.60,2.05,7.60,22.7,8.7,-6.51,-8.83,-4.55,...,0.02,33,17↗,10.8↗,15↗,-0.0566,-0.0702,-0.0653,-0.0996,-0.1050
178,3019,-5.20,28.10,200.00,28.10,39.8,18.0,31.50,47.40,19.80,...,1.19,2985,2587↗,2380↘,3293↘,0.0023,0.0150,0.0075,0.0038,-0.0098
179,8027,4.45,8.15,44.00,8.15,25.8,5.2,-45.70,-64.00,-29.80,...,2.43,925,2305↗,1574↘,2018↘,-0.0385,-0.0132,-0.0109,0.1490,0.4320
180,3465,15.00,3.22,26.40,3.22,27.3,12.5,-11.70,-17.10,-7.30,...,0.07,89,47.5↗,27.7↗,23↗,0.0567,0.0976,0.1030,0.2820,0.6090


In [400]:
#create rise_fall
df_data['rise_fall'] = (df_data['一個月累計漲跌幅']).map(lambda x : 1 if x >= 0 else 0)

In [401]:
for a in df_data:
    if type(df_data[a][0]) == str :
        
        print(f"{a} {df_data[a][0]}")

14 20M07
15 20Q1
58 776↗
59 688↗
60 728↗
73 20M07
74 20Q1
75 14高
76 1高
77 2高
78 2增→減
159 20W32
170 20M08
183 34.65↘
184 34.5↗
185 34.39↘
186 32.62↗
187 29.14↗
188 28.07↗
195 18841↘
196 19625↘
197 27961↘


In [402]:
#delete the feature which is not important
del df_data['14']
del df_data['15']
del df_data['73']
del df_data['74']
del df_data['159']
del df_data['170']

In [403]:
for a in df_data:
    if type(df_data[a][0]) == str :
        
        print(f"{a} {df_data[a][0]}")

58 776↗
59 688↗
60 728↗
75 14高
76 1高
77 2高
78 2增→減
183 34.65↘
184 34.5↗
185 34.39↘
186 32.62↗
187 29.14↗
188 28.07↗
195 18841↘
196 19625↘
197 27961↘


In [404]:
df_data['58'][0][-1:]

'↗'

In [405]:
#change str to float or int
df_data['new_58'] = (df_data['58']).map(lambda x : float(x[:-1]) if x[-1:]=='↗' else float(x[:-1])*-1)
del df_data['58']

In [406]:
df_data['new_59'] = (df_data['59']).map(lambda x : float(x[:-1]) if x[-1:]=='↗' else float(x[:-1])*-1)
del df_data['59']
df_data['new_60'] = (df_data['60']).map(lambda x : float(x[:-1]) if x[-1:]=='↗' else float(x[:-1])*-1)
del df_data['60']
df_data['new_183'] = (df_data['183']).map(lambda x : float(x[:-1]) if x[-1:]=='↗' else float(x[:-1])*-1)
del df_data['183']
df_data['new_184'] = (df_data['184']).map(lambda x : float(x[:-1]) if x[-1:]=='↗' else float(x[:-1])*-1)
del df_data['184']
df_data['new_185'] = (df_data['185']).map(lambda x : float(x[:-1]) if x[-1:]=='↗' else float(x[:-1])*-1)
del df_data['185']
df_data['new_186'] = (df_data['186']).map(lambda x : float(x[:-1]) if x[-1:]=='↗' else float(x[:-1])*-1)
del df_data['186']
df_data['new_187'] = (df_data['187']).map(lambda x : float(x[:-1]) if x[-1:]=='↗' else float(x[:-1])*-1)
del df_data['187']
df_data['new_188'] = (df_data['188']).map(lambda x : float(x[:-1]) if x[-1:]=='↗' else float(x[:-1])*-1)
del df_data['188']
df_data['new_195'] = (df_data['195']).map(lambda x : float(x[:-1]) if x[-1:]=='↗' else float(x[:-1])*-1)
del df_data['195']
df_data['new_196'] = (df_data['196']).map(lambda x : float(x[:-1]) if x[-1:]=='↗' else float(x[:-1])*-1)
del df_data['196']
df_data['new_197'] = (df_data['197']).map(lambda x : float(x[:-1]) if x[-1:]=='↗' else float(x[:-1])*-1)
del df_data['197']

In [407]:
for a in df_data:
    if type(df_data[a][0]) == str :
        
        print(f"{a} {df_data[a][0]}")
        


75 14高
76 1高
77 2高
78 2增→減


In [408]:
testdata = df_data.drop(labels=['75','76','77','78'],axis=1)

In [409]:
#plt.figure(figsize=(15,15))
#sns.heatmap(testdata.corr(),annot=True)

In [410]:
#find NaN
x =testdata.isnull().any()
IND = []
for index, value in x.items():
    if value==True:
        #print(f"Index : {index}, Value : {value}")
        IND.append(index)

In [411]:
for I in IND:
    testdata[I] = testdata[I].fillna(testdata[I].median())#把NaN用中位數取代

In [412]:
#cut data
df_train = testdata.loc[:150]
df_test = testdata.loc[151:]

In [413]:
X = df_train.drop(labels=['代號','rise_fall','一個月累計漲跌幅'],axis=1)
Y = df_train['rise_fall']

In [414]:
Base_Model = RandomForestClassifier(random_state=2,n_estimators=250,min_samples_split=22,oob_score=True)
Base_Model.fit(X, Y)
print('Base oob score :%.5f' %(Base_Model.oob_score_),'   LB_Public : 0.77751')

Base oob score :0.95364    LB_Public : 0.77751


In [415]:
X_Submit = df_test.drop(labels=['代號','rise_fall','一個月累計漲跌幅'],axis=1)

Base_pred = Base_Model.predict(X_Submit)

submit = pd.DataFrame({"代號": df_test['代號'],
                      "rise_fall":Base_pred.astype(int)})


### 上面是對漲跌做預測,再來是對漲跌幅度做預測
---

In [460]:
updown = testdata.drop(labels=['rise_fall'],axis=1)

In [461]:
max(updown['一個月累計漲跌幅'])

59.8

In [462]:
min(updown['一個月累計漲跌幅'])

-35.2

In [463]:
updown['一個月累計漲跌幅'] = (updown['一個月累計漲跌幅']).map(lambda x : int(x/7))

In [464]:
to_log_index = []
for x in updown:
    if max(updown[x])+abs(min(updown[x])) >=20:
        to_log_index.append(x)

    

In [465]:
for x in to_log_index:
    updown[x] = (updown[x]).map(lambda x : np.log10(x) if x > 0 else (-1*np.log10(abs(x)) if x<0 else 0))

In [466]:
updown

,代號,一個月累計漲跌幅,1,2,3,4,5,6,7,8,...,new_60,new_183,new_184,new_185,new_186,new_187,new_188,new_195,new_196,new_197
0,3.509337,0,2.453318,2.984977,2.453318,1.283301,1.230449,1.225309,1.320146,1.089905,...,2.862131,-1.539703,1.537819,-1.536432,1.513484,1.464490,1.448242,-4.275104,-4.292810,-4.446553
1,3.254064,-1,1.389166,2.283301,1.389166,1.733197,-0.154902,2.068186,2.201397,1.897077,...,0.930440,-1.896747,1.895312,-1.903036,1.907680,-1.891816,-1.953276,2.970347,2.924796,-3.137671
2,3.397245,0,0.243038,1.041393,0.243038,1.465383,1.264818,1.332438,1.563481,1.093422,...,0.154902,1.793092,-1.797752,-1.806316,1.830139,-1.810434,-1.822168,1.713491,-1.747412,-1.881385
3,3.541704,0,2.987219,2.896526,2.987219,1.245513,1.139879,1.173186,1.267172,1.064458,...,2.318063,-0.916454,0.918555,-0.920123,0.884229,-0.857332,0.865696,-5.055783,-5.068776,-5.062450
4,3.503655,3,1.654177,2.570543,1.654177,1.298853,1.198657,1.522444,1.622214,1.389166,...,1.324282,1.914343,1.897572,1.865874,1.804753,1.744293,1.707059,4.280351,-4.255176,4.178776
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,3.918607,0,0.880814,0.311754,0.880814,1.356026,0.939519,-0.813581,-0.945961,-0.658011,...,0.585027,-0.456366,-0.462398,-0.460898,-0.477121,-0.480007,-0.530200,1.230449,1.033424,1.176091
178,3.479863,0,1.448706,2.301030,1.448706,1.599883,1.255273,1.498311,1.675778,1.296665,...,-1.110590,1.850891,1.845408,-1.848620,1.850217,-1.856185,-1.894094,3.412796,-3.376577,-3.517592
179,3.904553,0,0.911158,1.643453,0.911158,1.411620,0.716003,-1.659916,-1.806180,-1.474216,...,0.143015,-1.749427,1.738146,1.737193,1.672005,1.576457,1.543447,3.362671,-3.197005,-3.304921
180,3.539703,2,0.507856,1.421604,0.507856,1.436163,1.096910,-1.068186,-1.232996,-0.863323,...,-1.397940,1.889862,1.873379,1.871047,1.805908,1.707315,1.630631,1.676694,1.442480,1.361728


In [467]:
df_train = updown.loc[:150]
df_test = updown.loc[151:]

In [468]:
X = df_train.drop(labels=['代號','一個月累計漲跌幅'],axis=1)
Y = df_train['一個月累計漲跌幅']

In [487]:
ud_Model = RandomForestClassifier(random_state=2,n_estimators=950,min_samples_split=15,oob_score=True)
ud_Model.fit(X, Y.astype('int'))
print('Base oob score :%.5f' %(ud_Model.oob_score_),'   LB_Public : 0.77751')

Base oob score :0.58940    LB_Public : 0.77751


In [488]:
test_x = df_train.drop(labels=['代號','一個月累計漲跌幅'],axis=1)
test_y = df_train['一個月累計漲跌幅']

In [489]:
test_y_predicted = ud_Model.predict(test_x)
print(test_y_predicted)


[ 0 -1  0  0  3  2  1  0  0  0  0  2  0  0 -2  0  1 -1  0 -1  1  0  1  1
 -1  0  0  0  0  0  0 -1 -1 -1  0  0 -1  4  0  0  2  0  0  1  0 -2  0  0
 -1  2  0  1  1  0 -1  0  2 -1 -1  0  1  0  0  0  0  0  1  0  1  0  0  0
  1  1  4  0  0  0  0  0 -1  1  1  1  1  0  0  4  2  4  0  0  0  3  2  0
  0  0  0  1  0  0  4 -1  0  0  2  0  3  1  3  0  1  0  0  0  0  0  1  2
 -1  0  1 -1  0  3 -1  1  1  0  0  0  0 -1  4  1  4  0  0  0  3  0 -1  1
 -1  1  1  1  0  1  0]


In [490]:
ans = []
for x in test_y:
    ans.append(x)
print(ans)


[0, -1, 0, 0, 3, 2, 6, -5, 0, 0, -4, 2, 5, 0, -2, 0, 1, -1, 0, -1, 8, 0, 1, 7, -1, 0, 0, 0, 0, 0, 0, -1, -4, -1, 0, 0, -3, 4, 0, 0, 2, 0, 0, 1, -2, -2, 0, 0, -1, 2, 0, 1, 7, 0, -2, 0, 2, -1, -3, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 4, 0, 0, 0, 0, -3, -1, 5, 1, 1, 1, 0, 0, 4, 2, 4, 0, 0, 0, 3, 2, 0, 0, 0, 0, 1, 0, 0, 4, -1, 0, 0, 2, 0, 3, 1, 3, 0, 2, 0, 0, 0, 0, 0, 3, 2, -2, 0, 1, -1, 0, 3, -1, 1, 1, 0, 0, 0, 0, -1, 4, 1, 4, 0, 0, 0, 3, 0, -1, 1, -1, 4, 1, 1, 0, 1, 0]


In [491]:
SS=0
for s in range(0,151):
    if test_y_predicted[s] == ans[s]:
        SS +=1
SS/151

0.8741721854304636